## 1. Understanding  Ray Sampling

In [1]:
# Importing Libraries and Packages
import torch
import math
import numpy as np

from pytorch3d.renderer import (
    FoVPerspectiveCameras,
    PointLights,
    look_at_view_transform,
    NDCGridRaysampler,
)

In [2]:
# Setting up PyTorch device
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
    print("WARNING: CPU only, this will be more slowly")

In [3]:
# Defining a batch of 10 cameras 
num_views: int = 10
azimuth_range: float = 180
elev = torch.linspace(0, 0, num_views)  # keep constant
azim = torch.linspace(-azimuth_range, azimuth_range, num_views) + 180.0
lights = PointLights(device=device, location=[[0.0, 0.0, -3.0]])

R, T = look_at_view_transform(dist=2.7, elev=elev, azim=azim)
cameras = FoVPerspectiveCameras(device=device, R=R, T=T)

In [4]:
# Defining a ray sampler
image_size = 64
volume_extent_world = 3.0

raysampler = NDCGridRaysampler(
    image_width=image_size,
    image_height=image_size,
    n_pts_per_ray=50,
    min_depth=0.1,
    max_depth=volume_extent_world,
)

In [5]:
# Letting the ray sampler know where our cameras are and in what directions they are pointing
# Getting the sampled rays and points into ray_bundle
ray_bundle = raysampler(cameras)

In [6]:
# Printing out information on ray bundle
print('ray_bundle origins tensor shape = ', ray_bundle.origins.shape)
print('ray_bundle directions shape = ', ray_bundle.directions.shape)
print('ray_bundle lengths = ', ray_bundle.lengths.shape)
print('ray_bundle xys shape = ', ray_bundle.xys.shape)

ray_bundle origins tensor shape =  torch.Size([10, 64, 64, 3])
ray_bundle directions shape =  torch.Size([10, 64, 64, 3])
ray_bundle lengths =  torch.Size([10, 64, 64, 50])
ray_bundle xys shape =  torch.Size([10, 64, 64, 2])


In [7]:
# Saving ray bundle
torch.save({
    'ray_bundle': ray_bundle
}, 'output/ray_sampling.pt')

## 2. Using Volume Sampling

In [8]:
# Importing Libraries and Packages
import torch
from pytorch3d.structures import Volumes
from pytorch3d.renderer.implicit.renderer import VolumeSampler

In [9]:
# Setting up PyTorch device
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
    print("WARNING: CPU only, this will be more slowly")

In [10]:
# Loading ray_bundle computed in the first step
checkpoint = torch.load('output/ray_sampling.pt')
ray_bundle = checkpoint.get('ray_bundle')

In [11]:
# Defining volume
batch_size = 10
densities = torch.zeros([batch_size, 1, 64, 64, 64]).to(device)
colors = torch.zeros(batch_size, 3, 64, 64, 64).to(device)
voxel_size = 0.1

volumes = Volumes(
    densities=densities,
    features=colors,
    voxel_size=voxel_size
)

In [12]:
# Defining volume_sampler
volume_sampler = VolumeSampler(volumes = volumes, sample_mode = "bilinear")
rays_densities, rays_features = volume_sampler(ray_bundle)

In [13]:
# Printing out the shapes for the densities and colors
print('rays_densities shape = ', rays_densities.shape)
print('rays_features shape = ', rays_features.shape)

rays_densities shape =  torch.Size([10, 64, 64, 50, 1])
rays_features shape =  torch.Size([10, 64, 64, 50, 3])


In [14]:
# Saving the densities and colors (to be able to use them in the next step)
torch.save({
    'rays_densities': rays_densities,
    'rays_features': rays_features
}, 'output/volume_sampling.pt')

## 3. Exploring the Ray Marcher

In [15]:
# Importing Libraries and Packages
import torch
from pytorch3d.renderer.implicit.raymarching import EmissionAbsorptionRaymarcher

In [16]:
# Loading densities and colors on rays from the second step
checkpoint = torch.load('output/volume_sampling.pt')
rays_densities = checkpoint.get('rays_densities')
rays_features = checkpoint.get('rays_features')

In [17]:
# Defining ray_marcher and passing the densities and colors 
# it will give us the rendered RGB values (image_features)
ray_marcher = EmissionAbsorptionRaymarcher()

image_features = ray_marcher(rays_densities = rays_densities, 
                             rays_features = rays_features)

In [18]:
# Printing out the image feature shape
print('image_features shape = ', image_features.shape)

image_features shape =  torch.Size([10, 64, 64, 4])


So, we have a batch of 10 images 64x64, and 4 channels: the first three - RGBs; the last one - the alpha channel (it represents whether the pixel is in the foreground or the background).

## 4. Reconstructing 3D Models from Multi-view Images

In [22]:
# Importing Libraries and Packages
import os
import sys
import time
import json
import glob
import torch
import math
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image

from pytorch3d.structures import Volumes
from pytorch3d.renderer import (
    FoVPerspectiveCameras,
    VolumeRenderer,
    NDCGridRaysampler,
    EmissionAbsorptionRaymarcher
)
from pytorch3d.transforms import so3_exp_map


# Specific files downloaded from 
# https://github.com/PacktPublishing/3D-Deep-Learning-with-Python/tree/main/chap5
from plot_image_grid import image_grid
from generate_cow_renders import generate_cow_renders

In [23]:
# Setting up PyTorch device
if torch.cuda.is_available():
    device = torch.device("cuda:0")
else:
    device = torch.device("cpu")
    print("WARNING: CPU only, this will be more slowly")

In [24]:
# Generating 40 cameras, images, and silhouette images with different angles
target_cameras, target_images, target_silhouettes = generate_cow_renders(num_views=40)

In [25]:
# Defining a ray sampler
render_size = 128
volume_extent_world = 3.0

raysampler = NDCGridRaysampler(
    image_width=render_size,
    image_height=render_size,
    n_pts_per_ray=150,
    min_depth=0.1,
    max_depth=volume_extent_world,
)

In [26]:
# Creating the ray marcher
raymarcher = EmissionAbsorptionRaymarcher()
renderer = VolumeRenderer(raysampler=raysampler, raymarcher=raymarcher,)

In [27]:
# Defining a class for encapsulating a volume
class VolumeModel(torch.nn.Module):
    def __init__(self, renderer, volume_size=[64]*3, voxel_size=0.1):
        super().__init__()
        self.log_densities = torch.nn.Parameter(-4.0 * torch.ones(1, *volume_size))
        self.log_colors = torch.nn.Parameter(torch.zeros(3, *volume_size))
        self._voxel_size = voxel_size
        self._renderer = renderer

    def forward(self, cameras):
        batch_size = cameras.R.shape[0]

        densities = torch.sigmoid(self.log_densities)
        colors = torch.sigmoid(self.log_colors)

        volumes = Volumes(
            densities=densities[None].expand(batch_size, *self.log_densities.shape),
            features=colors[None].expand(batch_size, *self.log_colors.shape),
            voxel_size=self._voxel_size,
        )
        return self._renderer(cameras=cameras, volumes=volumes)[0]

In [28]:
# Defining a Huber loss function
def huber(x, y, scaling=0.1):
    diff_sq = (x - y) ** 2
    loss = ((1 + diff_sq / (scaling ** 2)).clamp(1e-4).sqrt() - 1) * float(scaling)
    return loss

In [29]:
# Moving to the right device
target_cameras = target_cameras.to(device)
target_images = target_images.to(device)
target_silhouettes = target_silhouettes.to(device)

In [30]:
# Creating inastance of VolumeModel
volume_size = 128

volume_model = VolumeModel(
    renderer,
    volume_size=[volume_size] * 3,
    voxel_size=volume_extent_world / volume_size,
).to(device)

In [31]:
# Setting up the optimizer
lr = 0.1
optimizer = torch.optim.Adam(volume_model.parameters(), lr=lr)

batch_size = 10
n_iter = 300

In [32]:
# Running optimization iterrations
for iteration in range(n_iter):

    if iteration == round(n_iter * 0.75):
        print('Decreasing LR 10-fold ...')
        optimizer = torch.optim.Adam(
            volume_model.parameters(), lr=lr * 0.1
        )

    optimizer.zero_grad()
    batch_idx = torch.randperm(len(target_cameras))[:batch_size]

    # Sample the minibatch of cameras
    batch_cameras = FoVPerspectiveCameras(
        R=target_cameras.R[batch_idx],
        T=target_cameras.T[batch_idx],
        znear=target_cameras.znear[batch_idx],
        zfar=target_cameras.zfar[batch_idx],
        aspect_ratio=target_cameras.aspect_ratio[batch_idx],
        fov=target_cameras.fov[batch_idx],
        device=device,
    )

    rendered_images, rendered_silhouettes = volume_model(
        batch_cameras
    ).split([3, 1], dim=-1)

    sil_err = huber(
        rendered_silhouettes[..., 0], target_silhouettes[batch_idx],
    ).abs().mean()

    color_err = huber(
        rendered_images, target_images[batch_idx],
    ).abs().mean()

    loss = color_err + sil_err
    
    if iteration % 10 == 0:
        print(
            f'Iteration {iteration:05d}:'
            + f' color_err = {float(color_err):1.2e}'
            + f' mask_err = {float(sil_err):1.2e}'
        )

    # Take the optimization step.
    loss.backward()
    optimizer.step()

Iteration 00000: color_err = 2.67e-01 mask_err = 5.67e-01
Iteration 00010: color_err = 1.08e-01 mask_err = 3.57e-01
Iteration 00020: color_err = 3.20e-02 mask_err = 1.61e-01
Iteration 00030: color_err = 1.34e-02 mask_err = 7.79e-02
Iteration 00040: color_err = 9.94e-03 mask_err = 4.45e-02
Iteration 00050: color_err = 7.37e-03 mask_err = 3.09e-02
Iteration 00060: color_err = 6.26e-03 mask_err = 2.39e-02
Iteration 00070: color_err = 5.54e-03 mask_err = 1.89e-02
Iteration 00080: color_err = 5.89e-03 mask_err = 1.77e-02
Iteration 00090: color_err = 5.13e-03 mask_err = 1.57e-02
Iteration 00100: color_err = 4.85e-03 mask_err = 1.48e-02
Iteration 00110: color_err = 5.01e-03 mask_err = 1.43e-02
Iteration 00120: color_err = 4.88e-03 mask_err = 1.34e-02
Iteration 00130: color_err = 4.66e-03 mask_err = 1.25e-02
Iteration 00140: color_err = 4.77e-03 mask_err = 1.39e-02
Iteration 00150: color_err = 4.37e-03 mask_err = 1.11e-02
Iteration 00160: color_err = 4.50e-03 mask_err = 1.14e-02
Iteration 0017

In [ ]:
# Taking the final volumetric model and render images from new angles
def generate_rotating_volume(volume_model, n_frames=50):
    logRs = torch.zeros(n_frames, 3, device=device)
    logRs[:, 1] = torch.linspace(0.0, 2.0 * 3.14, n_frames, device=device)
    Rs = so3_exp_map(logRs)
    Ts = torch.zeros(n_frames, 3, device=device)
    Ts[:, 2] = 2.7
    frames = []
    print('Generating rotating volume ...')
    for R, T in zip(Rs, Ts):
        camera = FoVPerspectiveCameras(
            R=R[None],
            T=T[None],
            znear=target_cameras.znear[0],
            zfar=target_cameras.zfar[0],
            aspect_ratio=target_cameras.aspect_ratio[0],
            fov=target_cameras.fov[0],
            device=device,
        )
        frames.append(volume_model(camera)[..., :3].clamp(0.0, 1.0))
    return torch.cat(frames)



with torch.no_grad():
    rotating_volume_frames = generate_rotating_volume(volume_model, n_frames=7*4)

image_grid(rotating_volume_frames.clamp(0., 1.).cpu().numpy(), 
           rows=4, cols=7, rgb=True, fill=True)
plt.savefig('rotating_volume.png')
#plt.show()

Generating rotating volume ...


In [35]:
volume_model

VolumeModel(
  (_renderer): VolumeRenderer(
    (renderer): ImplicitRenderer(
      (raysampler): NDCMultinomialRaysampler()
      (raymarcher): EmissionAbsorptionRaymarcher()
    )
  )
)